In [ ]:
import numpy as np
import math
import gc
import sys
import multiprocessing
from joblib import Parallel,delayed
import time
import numba
from numba import jit,cuda
import timeit
import random

In [ ]:

#binary sum has been replaced with the xor operation ^
def vectorbinarysum(v1,v2): #sums binary vectors
    if len(v1) != len(v2):
        return
    n = len(v1)
    sum =[]
    for i in range(n):
       sum.append(v1[i]^v2[i])
    return sum

def decimalToBinary(n,x):
    a = bin(n).replace("0b", "")
    return a.zfill(x)

def combinatorial(n,r):
    return math.comb(n,r)
    #return math.factorial(n)/(math.factorial(r)*math.factorial(n-r))

def combsum(m,r): #sums combinatorials from (m,0) to (m,r), gives dimension of rm code
    sum = 0
    for i in range(r+1):
        sum+= combinatorial(m,i)
    return sum
generator = 1 #intialising generator matrix
#fullcode can be y, yy or any other. Each results in a separate case, details below

def code_generator(r,m,fullcode):#generates code based on requirement
    n = 2**m
    basis = [[0 for x in range(n)] for y in range(m)] #initialising the set of boolean funtions v_1 to v_m except unit vector 1
    for i in range(n):
            a = list(decimalToBinary(i,m))
            for j in range(m):
                basis[j][i] = int(a[j])
    k = int(combsum(m,r))
    global generator
    generator = [[0 for x in range(n)] for y in range(m+1)] # initialising generator matrix
    for i in range(n):
        for j in range(1,m+1):
            generator[j][i] = basis[j-1][i]
    for i in range(n):
        generator[0][i]=1
    # Below two functions are to print all combination
    # of size r in an array of size n
    # Used for obtaining row of genereartor matrix
    def printCombination(arr, m, r, n):
        data = [0]*r;
        combinationUtil(arr, data, 0,
                            m - 1, 0, r,n);
        # arr[] ---> Input 
        # Array data[] ---> Temporary array to store current combination
        # start & end ---> Starting and Ending indexes in arr[]
        # index ---> Current index in data[]
        # r ---> Size of a combination to be printed
        # n is size of codeword
    def combinationUtil(arr, data, start,
                            end, index, r,n):
        if (index == r):
            data2 = [1 for x in range(n)]
            for j in range(r):
                for x in range(n):
                    data2[x] *= data[j][x]
            global generator
            generator.append(data2)
            return;
        i = start;
        while(i <= end and end - i + 1 >= r - index):
            data[index] = arr[i];
            combinationUtil(arr, data, i + 1,
                            end, index + 1, r,n);
            i += 1;
    #printcombination and combinationUtil are modifications of code obtained from
    #https://www.geeksforgeeks.org/print-all-possible-combinations-of-r-elements-in-a-given-array-of-size-n/

    
    for i in range(2,r+1):
        printCombination(basis, m, i, n)#i instead of r fixed bug
    #It appears that as r,m increases printCombination 
    #needs to be called more times to work properly
    #Don't know why, in case of error here increase range from 10 until error is resolved
    k2 = int(combinatorial(m,r))
    if fullcode == 'y': #generates the complete r,m code
        rmcode = [[0 for i in range(n)] for j in range(2**k)]
        coefficient = [1 for i in range(2**k)]
        #coefficeint stores all binary vectors from 0 to 2**k, used to generate all possible linear combinations of the
        #generator matrix
        for i in range(2**k):
            a = list(decimalToBinary(i,k))
            for j in range(k):
                a[j] = int(a[j]) 
            coefficient[i] = a
        for i in range(2**k):
            for j in range(n):
                for t in range(k):
                    rmcode[i][j] = rmcode[i][j]^(coefficient[i][t]*generator[t][j])
        return rmcode
    elif fullcode == 'yy': #find codewords of r,m code not in r-1, m code
        rmcode = [[0 for i in range(n)] for j in range(2**k - 2**(k-k2))]
        coefficient = []
        for i in range(2**k):
            a = list(decimalToBinary(i,k))
            for j in range(k):
                a[j] = int(a[j])
           
            if a[k-k2::k] != [0 for i in range(k2)]:
                coefficient.append(a)
        for i in range(2**k - 2**(k-k2)):
            for j in range(n):
                for t in range(k):
                    rmcode[i][j] =rmcode[i][j]^(coefficient[i][t]*generator[t][j])
        return rmcode 
    elif fullcode == 1: #If you just wish to obtain generator array
        return
    else: #finds the codewords with the basis as all basis vectors of r,m RM code (r,m are arguments passed to the function) 
        #minus the basis vectors of the r-1,m RM code
        rmcode = [[0 for i in range(n)] for j in range(2**k2)]
        print(1,2**k2,sys.getsizeof(rmcode))
        coefficient = [1 for i in range(2**k2)]
        for i in range(2**k2):
            a = list(decimalToBinary(i,k2))
            for j in range(k2):
                a[j] = int(a[j]) 
            coefficient[i] = a
        print(1)
        
        for i in range(2**k2):
            
            for j in range(n):
                for t in range(k2):
                    rmcode[i][j] = rmcode[i][j]^(coefficient[i][t]*generator[t + k - k2][j])
            
        return rmcode

def code_weight(code): #calculating weight of a code
    n =len(code[0])
    weight = [0 for i in range(n+1)]
    for i in range(len(code)):
        sum=0
        for j in range(n):
            sum+= code[i][j]
        weight[sum] +=1
    return weight

def coset(rmcode,vector): #returns modified rmcode
    for j in range(len(rmcode)):
        rmcode[j] = vectorbinarysum(rmcode[j],vector)
    return rmcode
def cosets(rmcode,vectors,i,n1):
    coset = [1 for j in range(n1)]
    for j in range(n1):
        coset[j] = vectorbinarysum(rmcode[j],vectors[i])
    return coset
def coset2(rmcode,vectors):#does not modify argument rmcode
    coset = [1 for j in range(len(rmcode))]
    for j in range(len(rmcode)):
        coset[j] = vectorbinarysum(rmcode[j],vectors)
    return coset
def allcosets(rmcode,vectors):
    n1 = len(rmcode) # number of codeword
    n3 = len(vectors) #number of vectors, each vector will generate a unique coset
    fullcoset = [1 for i in range(n3)]
    for i in range(n3):
        fullcoset[i] = cosets(rmcode,vectors,i,n1)        
    return fullcoset

def squarepolynomial(array):
    return multiply(array,array)

def lemma1(weight):
    enumerator = np.array([])
    for i in range(len(weight)):
        enumerator = add(enumerator,squarepolynomial(weight[i]))
    return enumerator

def binarytodecimal(binary):
    return int(binary,2)


def sumcodes(c1,c2): #returns the code consisting of the sum of codewords of two linear codes
    n1 = len(c1)
    n2 = len(c2)
    codesum = []#TO OPTIMIZE
    for i in range(n1):
        for j in range(n2):
            codesum.append(vectorbinarysum(c1[i],c2[j]))#not worth using bitwise_xor directly here, maybe try in the funuction
    res = [] # from here till return res, function to remove duplicates in a list obtained from
    for i in codesum: #https://www.geeksforgeeks.org/python-ways-to-remove-duplicates-from-list/
        if i not in res:
            res.append(i)
    return res


In [ ]:
@jit(nopython = True)
def bintodec(binary):
    decimal = 0
    length = len(binary)
    for i in range(len(binary)):
        decimal += binary[length - i - 1]*2**i
    return decimal
    
@jit(nopython = True)
def energy(weight_size, codeword):
    sum = 0
    for i in codeword:
        sum+=i
    return abs(weight_size-sum)

@jit
def efficient_metropolis_sampler(fixed_codeword, beta, weight_size,M,n,r,coefficient,shift): #function to randomly sample codewords
    c_prev = fixed_codeword
    for i in range(M):
        #current = time.time()
        flag = 0
        while flag == 0:
            A = np.random.randint(2, size = (n-r,n))
            if np.linalg.matrix_rank(A) == n-r:
                flag = 1
        b = np.random.randint(2, size = n)
        temp = np.matmul(coefficient,A)
        temp = temp%2
        positions = []
        for j in range(len(temp)):
            temp[j] = temp[j]^b
            decimal = 0
            length = len(temp[j])
            for k in range(len(temp[j])):
                decimal += temp[length - k - 1]*2**k
            positions.append(decimal)
            #positions.append(binarytodecimal(''.join(map(str,temp[j]))))
        c_incidence = np.zeros(2**n, dtype = int)
        for j in positions:
            c_incidence[2**n - 1 - j] = 1 
        #GENERATE INCIDENCE VECTOR 
        c_temp = c_prev^c_incidence
        energy1 = 0
        for m in c_prev:
            energy1+=m
        energy1 = abs(weight_size- energy1)
        energy2 = 0
        for m in c_temp:
            energy2+=m
        energy2 = abs(weight_size- energy2)
        shift_threshold = math.exp(beta*(energy1 - energy2))
        # if 1 < shift_threshold:
        #     shift_threshold = 1
        #NOT NEEDED
        if shift[i] <= shift_threshold:
            c_prev = c_temp
        #print(time.time() - current)
    return c_prev


        
        

In [ ]:
import decimal

In [ ]:
import mpmath

In [ ]:

def metropolis_sampler(fixed_codeword, beta, weight_size,M,n,r,coefficient):#,precision):
    c_prev = fixed_codeword
    #mp.dps=precision
    for i in range(M):
        #current = time.time()
        #Finding c_temp vector at random can be function of its own - didnt work
        flag = 0
        while flag == 0:
            A = np.random.randint(2, size = (n-r,n))
            if np.linalg.matrix_rank(A) == n-r:
                flag = 1
        b = np.random.randint(2, size = n)
        temp = np.matmul(coefficient,A)
        temp = temp%2
        positions = []
        for j in range(len(temp)):
            temp[j] = temp[j]^b
            positions.append(bintodec(temp[j]))
            #positions.append(binarytodecimal(''.join(map(str,temp[j]))))
        c_incidence = np.zeros(2**n, dtype = int)
        for j in positions:
            c_incidence[2**n - 1 - j] = 1 
        #GENERATE INCIDENCE VECTOR 
        c_temp = c_prev^c_incidence
        #beta = mpf(beta)
        if (energy(weight_size,c_prev) - energy(weight_size, c_temp))>=0:
            shift_threshold=1#*10**precision
        else:
            shift_threshold = math.exp(beta*(energy(weight_size,c_prev) - energy(weight_size, c_temp)))#*10**precision
        # if 1 < shift_threshold:
        #     shift_threshold = 1
        #NOT NEEDED
        shift = random.uniform(0,1)
        #shift = random.randint(0,10**precision)
        # if i%100==0:
        #     print(shift_threshold)
        if shift <= shift_threshold:
            c_prev = c_temp
        # if i==(M-1):
        #     print(time.time() - current)
    return c_prev


In [ ]:
# @jit(nopython = True)
# def add(x,y):
#     sum = 0
#     array = np.zeros(10)
#     for i in range(10):
#         array[i]= sum + i + x + y
#         array[i] = bintodec(array)
#     return array, energy(5,array)

# add(1,1)

The subspace H, its elements correspond to the ones of the incidence vector

In [ ]:
M = 10000 #Number of iterations of metropolis algorithm
M2 = 100 #averaging over M2 samples of estimator
n = 4
r = 2
code_generator(r,n,1)
g = generator
k = combsum(n,r)
print(n,r,M,M2,k)
#mp.dps = 10
coefficient = [1 for i in range(2**(n-r))]
        #coefficeint stores all binary vectors from 0 to 2**k, used to generate all possible linear combinations of the
        #generator matrix
for i in range(2**(n-r)):
    a = list(decimalToBinary(i,n-r))
    for j in range(n-r):
        a[j] = int(a[j]) 
    coefficient[i] = a
coefficient

In [ ]:
M = 10000 #Number of iterations of metropolis algorithm
M2 = 100 #averaging over M2 samples of estimator
n = 4
r = 2
code_generator(r,n,1)
g = generator
k = combsum(n,r)
print(n,r,M,M2,k)
#mp.dps = 10
coefficient = [1 for i in range(2**(n-r))]
        #coefficeint stores all binary vectors from 0 to 2**k, used to generate all possible linear combinations of the
        #generator matrix
for i in range(2**(n-r)):
    a = list(decimalToBinary(i,n-r))
    for j in range(n-r):
        a[j] = int(a[j]) 
    coefficient[i] = a
weights = []
for weight_size in [0]: #loop to range over different weights; range(0,2**(n-2) + 1)
    #weight_size = 2*i
    print(weight_size)
    previous = 0
    estimate = 2**k
    beta = 0
    count = 0
    delta = 0.001
    while abs(estimate - previous) > delta:
        #The calculation of Y can be done in parallel
        Y = 0
        current = time.time()
        samples=[]
        for j in range(M2): #Number of random variables Y
            #current2 = time.time()
            random_message = np.random.randint(2, size = k)
            fixed_codeword = np.matmul(random_message,g)
            fixed_codeword = fixed_codeword%2
            sampled_codeword =  metropolis_sampler(fixed_codeword, beta, weight_size,M,n,r,coefficient)#,1000)
            #sampled_codeword =  efficient_metropolis_sampler(fixed_codeword, beta, weight_size,M,n,r,coefficient,shift)
            Y += math.exp((-1)*(energy(weight_size,sampled_codeword))*(1/n))
            sum = 0
            for i in sampled_codeword:
                sum+=i
            samples.append(sum)
            #print('inner', time.time() - current2)
        print('outer', time.time() - current)
        previous = estimate
        estimate = (estimate*Y)/M2
        beta += 1/n
        #count += 1
        #if estimate <0.5:
        #    break
        print(abs(estimate - previous), estimate, samples)#samples here
        # if count == 15:
        #     break 
        # if estimate < 0.5:
        #     break
    weights.append(estimate)
    print(weights)
weights
